In [676]:
# Maximizing the payout of a parlay
# Authors: Dejon Wright and Mela Coffey

from datetime import date, timedelta, datetime
from dateutil import parser
import pytz
import requests
import pandas as pd
import numpy as np
import os
import gurobipy as gp
from gurobipy import GRB
import math
from scipy.stats import poisson

In [677]:
class MLBOdds():
    API_KEY = os.environ.get("THE_ODDS_API", "")
    MARKETS = [
        # "batter_hits",
        # "batter_home_runs",
        # "batter_rbis", # gets infeasible 
        # "batter_singles",
        # "batter_doubles", 
        # "batter_runs_scored",
        "pitcher_strikeouts", # 1
        # "batter_total_bases", 
        "batter_hits_runs_rbis" # 4
    ]
    ODDS_FORMAT = 'decimal'
    BOOKMAKERS = 'draftkings'
    REGIONS = 'us'
    WEEKS = 3
    
    def __init__(self) -> None:
        self.oddsdf = pd.concat([self.get_odds(event_id=event) for event in self.get_events()], axis=0, ignore_index=True)
        self.statsdf = self.get_stats()
        
    def get_events(self):
        response = requests.get(
            f'https://api.the-odds-api.com/v4/sports/baseball_mlb/scores', 
            params={
                'api_key': self.API_KEY,
            }
        )
        if response.status_code != 200:
            print(f'Failed to get sports: status_code {response.status_code}, response body {response.text}')
        else:
            utc = pytz.UTC
            now = utc.localize(datetime.now())
            tomorrow = now + timedelta(days=1)
            data = response.json()
            return [event["id"] for event in data if not event["scores"] and (now <= parser.parse(event['commence_time']) <= tomorrow)]
        return []

    def get_odds(self, event_id: str):
        response = requests.get(
            f'https://api.the-odds-api.com/v4/sports/baseball_mlb/events/{event_id}/odds/', 
            params={
                'api_key': self.API_KEY,
                'regions': self.REGIONS,
                'bookmakers': self.BOOKMAKERS,
                'oddsFormat': self.ODDS_FORMAT,
                'markets': ",".join(self.MARKETS)
            }
        )

        if response.status_code != 200:
            print(f'Failed to get sports: status_code {response.status_code}, response body {response.text}')

        elif not response.json()['bookmakers']:
            # No odds available, return empty frame
            return pd.DataFrame()
        else:
            data = []
            home = response.json()['home_team'].split(" ", 1)[1]
            away = response.json()['away_team'].split(" ", 1)[1]
            teams = f"{away}@{home}"
            dk = response.json()['bookmakers'][0]
            for market in dk["markets"]:
                outcomes = market['outcomes']
                key = market['key']
                for i in range(0, len(outcomes), 2):
                    d = {}
                    d["Name"] = f'{outcomes[i]["description"]}'
                    d["Teams"] = teams
                    d["Game"] = event_id
                    d[f"{key}_odds_point"] = outcomes[i]["point"]
                    d[f"{key}_over"] = outcomes[i]["price"]
                    d[f"{key}_under"] = outcomes[i + 1]["price"]
                    data.append(d)
            base_frame = pd.DataFrame(data)
            column_map = {col: "sum" for col in base_frame.columns}
            column_map["Name"] = "first"
            column_map["Game"] = "first"
            column_map["Teams"] = "first"
            base_frame = base_frame.groupby(['Name', 'Game'], as_index=False).aggregate(column_map)
            base_frame.fillna(0,inplace=True)
            return base_frame

    def get_stats(self):
        def get_point_key(search_key):
            search_key = search_key.rsplit("_", 1)[0]
            return f"{search_key}_odds_point"
        def get_stat_total(search_key):
            search_key = search_key.rsplit("_", 1)[0]
            total = 1
            stat_total_map = {
                "batter_hits_runs_rbis": 3
            }
            if search_key in stat_total_map:
                total = stat_total_map[search_key] 
            return total
        def get_stat_key(odds_column: str, search_key_only=False):
            search_key = odds_column.rsplit("_", 1)[0]
            if search_key_only:
                return search_key
            odds_stat_map = {
                "batter_hits": 'Hits', 
                "batter_singles": 'Singles', 
                "batter_doubles": 'Doubles', 
                "batter_home_runs": 'HomeRuns', 
                "batter_rbis": 'RunsBattedIn',
                "pitcher_strikeouts": "PitchingStrikeouts",
                "batter_runs_scored": "Runs",
                "batter_hits_runs_rbis": ['Hits', "Runs", 'RunsBattedIn'],
                "batter_total_bases": ['Singles', 'Doubles', 'Triples', 'HomeRuns']
            }
            return odds_stat_map[search_key]
        
        today = date.today()
        week_prior =  today - timedelta(weeks=self.WEEKS)

        df = pd.read_csv('~/Development/sports-playground/PlayerGame.2023.csv')

        df['DateTime'] = pd.to_datetime(df['DateTime']).dt.date
        df = df[df['Started'] == 1]
        pitchers = df[df['PositionCategory'] == 'P']
        batters = df[df['PositionCategory'] != 'P']
        batters = batters[batters['DateTime'].between(week_prior, today)]
        df = pd.concat([pitchers, batters])

        df = df[[
                'Name', 
                'Runs', 
                'Hits', 
                'Singles', 
                'Doubles', 
                'Triples', 
                'HomeRuns', 
                'RunsBattedIn',
                'PitchingStrikeouts'
            ]]

        df.sort_values(by=["Name"])

        stats_data = []
        for _, row in self.oddsdf.iterrows():
            player = row["Name"]
            temp_df = df[df.Name == player]
            d1 = {
                "Name": player,
                "Game": row["Game"],
                "Teams": row["Teams"]
            }
            d2 = {}
            for col in row.keys():
                if any(word in col for word in ["Name", "Game", "Teams", "point"]):
                    continue
                if math.isnan(row[get_point_key(col)]):
                    continue
                cols = get_stat_key(col)
                if get_stat_key(col, True) == 'batter_total_bases':
                    x = (temp_df[cols[0]] + (temp_df[cols[1]] * 2) + (temp_df[cols[2]] * 3) + (temp_df[cols[3]] * 4)).mean()
                else:
                    x = temp_df[cols].mean() if isinstance(cols, str) else temp_df[cols].values.sum(axis=1).mean()
                x = 0 if np.isnan(x) else x
                mu = math.ceil(row[get_point_key(col)]) 
                ps = [poisson.pmf(i, x) for i in range(20)]
                p = sum(ps[:mu]) if "under" in col else sum(ps[mu:])
                d2[col] = np.float16(p)
            stats_d = {**d1, **d2}       
            stats_data.append(stats_d)
            
        stats_df = pd.DataFrame(stats_data)
        self.oddsdf = self.oddsdf.loc[:,~self.oddsdf.columns.str.endswith("_odds_point")]
        return stats_df



In [678]:
mlbdata = MLBOdds()

/var/folders/vx/ykltqjpn1dl8cbk5z64tn0gm0000gn/T/ipykernel_60511/3053007940.py:118: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('~/Development/sports-playground/PlayerGame.2023.csv')
/var/folders/vx/ykltqjpn1dl8cbk5z64tn0gm0000gn/T/ipykernel_60511/3053007940.py:160: RuntimeWarning: Mean of empty slice.
  x = temp_df[cols].mean() if isinstance(cols, str) else temp_df[cols].values.sum(axis=1).mean()
/Users/dejon/miniconda3/envs/betting/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/vx/ykltqjpn1dl8cbk5z64tn0gm0000gn/T/ipykernel_60511/3053007940.py:160: RuntimeWarning: Mean of empty slice.
  x = temp_df[cols].mean() if isinstance(cols, str) else temp_df[cols].values.sum(axis=1).mean()
/Users/dejon/miniconda3/envs/betting/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: inval

In [679]:
# Prepare data
copy_statsdf = mlbdata.statsdf.copy().set_index(['Name'])
copy_oddsdf = mlbdata.oddsdf.copy().set_index(['Name'])
table_df = pd.concat([copy_statsdf, copy_oddsdf]).stack().groupby(level=[0,1]).apply(tuple).unstack()
table_df = table_df.loc[:, ~table_df.columns.str.contains('^Unnamed')]

# Grab games
games = table_df['Game'].to_dict()
teams = { k: v[0] for k, v in table_df['Teams'].to_dict().items() } 
table_df.pop("Game")
table_df.pop("Teams")
games_set = set()
# remove tuple
for k, v in games.items():
    games[k] = v[0]
    games_set.add(v[0])

games_set = list(games_set)
table_dict = table_df.stack().to_dict()
odds, frequencies, projections, player_combos = [], [], [], []
games_map = [ [0] * len(games_set) for i in range(len(table_dict))]
game_indices_map = { games_set[i]: i for i in range(len(games_set)) }
player_counter = 0
game_indices = []
for k, v in table_dict.items():
    odds.append(v[1] if len(v) >= 2 else 0.0)
    frequencies.append(v[0])
    pc_tuple = (k[0], k[1], teams[k[0]])
    player_combos.append(pc_tuple)
    player_game = games[k[0]]
    game_index = game_indices_map[player_game]
    game_indices.append(game_index)
    games_map[player_counter][game_index] = 1
    player_counter += 1
size = len(player_combos)

In [680]:
min_legs = 2
max_legs = 20
max_risk = .45


In [681]:
m = gp.Model('MLB Player Props Builder')
m.params.Threads = 10
m.params.NonConvex = 2
m.params.MIPFocus = 1

selected = []
for i in range(size):
    selected.append(m.addVar(vtype=GRB.BINARY, name='-'.join(str(i) for i in player_combos[i])))

m.addConstr(
    min_legs <= gp.quicksum(selected), "min_legs" # <= self.max_legs
)

m.addConstr(
    max_legs >= gp.quicksum(selected), f"max_legs"
)

for i in range(size):
    m.addConstr(
        0 <= selected[i] * (frequencies[i] - max_risk), f"risk_{i}"
    )

for i in range(len(games_set)):
    m.addConstr(
        1 >= gp.quicksum([games_map[player][i] * selected[player] for player in range(size)]), f"uniq_games_{i}"
    )
for i in range(size):
    m.addConstr(
        0 <= selected[i] * (odds[i] - 1)
    )
# m.setObjective(
#     gp.quicksum([frequencies[i] * selected[i] for i in range(size)]), GRB.MAXIMIZE
# )
m.ModelSense = GRB.MAXIMIZE
m.setObjectiveN(
    expr=gp.quicksum([odds[i] * selected[i] for i in range(size)]),
    index=1,
    priority=0,
    weight=1
)
m.setObjectiveN(
    expr=gp.quicksum([frequencies[i] * selected[i] for i in range(size)]),
    index=0,
    priority=0,
    weight=500
)

# run optimization engine
m.optimize()
selected_odds = []
best_player_combos = []
nSolutions  = m.SolCount
for s in range(nSolutions):
    m.params.SolutionNumber = s
    print(f"Solution {s}")
    for i in range(len(selected)):
        try:
            if selected[i].Xn == 1:
                print(f"{player_combos[i]} {frequencies[i]} {odds[i]}")
        except:
            print('INFEASIBLE SOLUTION')


Set parameter Threads to value 10
Set parameter NonConvex to value 2
Set parameter MIPFocus to value 1
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 679 rows, 668 columns and 2672 nonzeros
Model fingerprint: 0xfa7eb953
Variable types: 0 continuous, 668 integer (668 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [8e-02, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
----------------------------------------------------------------